Este script tiene el fin de darle el formato correcto a los datos de las estaciones del SMN para poder aplicar el control de cálidad de RClimdex. Los requisitos son:
- Tiene que ser un txt, ASCII
- Columnas: año, mes, dia, prcp, tmax, tmin (separados por mas de un espacio)
- Código de datos faltantes -99.9 

## Librerias


In [6]:
import pandas as pd
import datetime
import numpy as np
from astropy.io import ascii
from astropy.table import Table


In [7]:
def formato_rclimdex(df):
    
    #Columnas: año, mes, día, pp, tmax, tmin
   
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    
    df = df[['year', 'month', 'day',  'pp', 'tmax', 'tmin']]
    df.pp = df.pp.replace('', '0')
    df[['tmax','tmin','pp']] = df[['tmax','tmin','pp']].replace(' ','0') #Reemplazo vacios por '0'
    df[['tmax','tmin','pp']] = df[['tmax','tmin','pp']].replace('S/D','-99.9') #Reemplazo S/D por '-1'
    df[['tmax','tmin','pp']] = df[['tmax','tmin','pp']].astype(float) #Paso a int
    return df
    
    

def pad(df):
    
    i=str(df.date.values[0]) #Primer fecha
    f=str(df.date.values[-1]) #Ultima fecha
    
    #array de fechas desde i hasta f con frecuencia diaria
    #comp = pd.to_datetime(pd.date_range(i, f, freq='D')).dt.date
    comp = np.arange(i, f, dtype='datetime64[D]')
    #Comparo el largo del array creado antes con el dataframe
    print('La diferencia de fechas es de '+str(len(comp)-len(df))+'\n')
    
    #Borro las fechas duplicadas
    df_comp = df.set_index('date')
    df_comp = df_comp.loc[~df_comp.index.duplicated(), :]
    #Reindexeo con la serie temporal completa llenando los vacios con 'S/D'
    df_comp = df_comp.reindex(comp,fill_value='S/D')
    
    #Cambio 
    df_comp['date']=comp

 
    return df_comp

def csv_asciitxt(csv_path):
    # reading csv file
    text = open(csv_path, "r")

    # joining with space content of text
    text = ' '.join([i for i in text])  

    # replacing ',' by space
    text = text.replace(",", "    ")  

    #displaying result
    return text

## Constantes

In [8]:
#Datos a leer
DATOS1='/home/dalia.panza/Proy_IAI/Datos/SMN/Exp186974.xlsx'
DATOS2='/home/dalia.panza/Proy_IAI/Datos/SMN/Exp186974_Parte2.xlsx'
DATOS_BRENIAS = '/home/dalia.panza/Proy_IAI/Datos/SMN/NH0416.xls'

#Donde se guardaran las salidas
SALIDAS='/home/dalia.panza/Proy_IAI/Salidas/SMN/'

#Vector con el nombre de todas las estaciones 
EST=['Santiago del Estero', 'Villa Maria del Rio Seco', 'Cordoba aero',
     'Cordoba observatorio','Escuela de Aviación militar', 'Pilar observatorio', 'Villa Dolores aero',
     'Río Cuarto aero', 'San Luis aero', 'Laboulaye aero', 'Villa Reynolds aero', 'Marco Juarez aero',
     'Termas de Rio Hondo', 'Chamical', 'Roque Saenz Peña aero', 'Santa Rosa de Conlara aero', 'Las Breñas']

#Vector con el nombre de todas las variables brindadas
COLS=['station','date','tmax','tmin','tmedia','pp', 'heliofania','direc_vmax','int_vmax', 'int_vmedio', 'trocio_media', 'pmedia', 
            'pmedia_nm', 'hr_media', 'nubosidad', 'ndatos']

## Lectura de datos

In [10]:
#Lectura de datos 
df1 = pd.read_excel(DATOS1,0)
df2 = pd.read_excel(DATOS2)

#Renombro las columnas
df1.columns=COLS
df2.columns=COLS

#Cambio la clase de las fechas a datetime
df1['date'] = pd.to_datetime(df1['date']).dt.date
df2['date'] = pd.to_datetime(df2['date']).dt.date

#df['date']=pd.to_datetime(df['date'], format='%d/%m/%Y').dt.dat
#df1['date']=pd.to_datetime(df['date'], format='%d/%m/%Y')
#df2['date']=pd.to_datetime(df['date'], format='%d/%m/%Y')

df=df2.append(df1, ignore_index=True)
#La estacion 10034 cierra el 8/9/1998 y abre la estación 17970 en su lugar. 
#Entonces le cambio el numero de estación 10034 --> 17970 para facilitar el 
#analisis (como si fueran la misma estación)

df.loc[df.station==10034,"station"]= 17970

#Datos de Las Breñas
brenias = pd.read_excel(DATOS_BRENIAS,0)


#Las Breñas es una estación del INTA, por eso está a parte. Me quedó con las columnas 0(fecha), 2(tmax)
#3(tmin) y 11(precip)
brenias = brenias.iloc[:, [0,2,3,11]]

#Le cambio el nombre a las columnas
brenias.columns = ['date', 'tmax', 'tmin', 'pp']
brenias['date']= pd.to_datetime(brenias['date']).dt.date
#Separo los datos por estación almacenandolos en un diccionario
estaciones = {EST[0]:df[df.station.eq(10062)],
              EST[1]:df[df.station.eq(10082)],
              EST[2]:df[df.station.eq(10100)],
              EST[3]:df[df.station.eq(10105)],
              EST[4]:df[df.station.eq(10107)],
              EST[5]:df[df.station.eq(10111)],
              EST[6]:df[df.station.eq(10117)],
              EST[7]:df[df.station.eq(10138)],
              EST[8]:df[df.station.eq(10139)],
              EST[9]:df[df.station.eq(10148)],
              EST[10]:df[df.station.eq(10335)],
              EST[11]:df[df.station.eq(10369)],
              EST[12]:df[df.station.eq(10458)],
              EST[13]:df[df.station.eq(10476)],
              EST[14]:df[df.station.eq(17970)],
              EST[15]:df[df.station.eq(18030)],
              EST[16]:brenias}


In [11]:
#Si hay saltos en la serie temporal uso la función pad para completarla 
for key in estaciones.keys():
    print(key)
    estaciones[key]=pad(estaciones[key])
       
    estaciones[key]=formato_rclimdex(estaciones[key])
    print(estaciones[key])
    
    estaciones[key].to_csv(SALIDAS+'Formato_RClimdex/'+key+'.txt', index=False, header=False, sep='\t')

Santiago del Estero
La diferencia de fechas es de 127

            year  month  day    pp  tmax  tmin
date                                          
1931-01-01  1931      1    1   8.0  35.4  19.7
1931-01-02  1931      1    2   0.0  35.8  20.7
1931-01-03  1931      1    3   0.0  37.7  23.9
1931-01-04  1931      1    4   0.0  39.5  25.8
1931-01-05  1931      1    5  20.1  29.0  22.8
...          ...    ...  ...   ...   ...   ...
2021-09-26  2021      9   26   0.0  36.2   9.9
2021-09-27  2021      9   27   0.0  34.8  18.0
2021-09-28  2021      9   28   0.0  28.4  18.5
2021-09-29  2021      9   29   0.0  22.2  17.0
2021-09-30  2021      9   30   0.0  34.4  19.0

[33146 rows x 6 columns]


/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Villa Maria del Rio Seco
La diferencia de fechas es de 230

            year  month  day    pp  tmax  tmin
date                                          
1931-01-01  1931      1    1   0.0  28.4  15.1
1931-01-02  1931      1    2   0.0  32.3  17.0
1931-01-03  1931      1    3   0.0  34.3  21.5
1931-01-04  1931      1    4  27.2  37.6  22.5
1931-01-05  1931      1    5   2.3  25.5  19.0
...          ...    ...  ...   ...   ...   ...
2021-09-26  2021      9   26   0.0  33.6   6.6
2021-09-27  2021      9   27   0.0  29.0  12.2
2021-09-28  2021      9   28   0.0  22.2  14.2
2021-09-29  2021      9   29   0.0  27.7  14.9
2021-09-30  2021      9   30  10.0  36.3  18.6

[33146 rows x 6 columns]


/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Cordoba aero
La diferencia de fechas es de 0

            year  month  day   pp  tmax  tmin
date                                         
1956-02-01  1956      2    1  0.0  24.3  13.9
1956-02-02  1956      2    2  0.0  28.6  11.4
1956-02-03  1956      2    3  0.0  29.6  11.5
1956-02-04  1956      2    4  0.0  33.2  20.8
1956-02-05  1956      2    5  4.0  21.4  12.0
...          ...    ...  ...  ...   ...   ...
2021-09-26  2021      9   26  0.0  26.6   9.0
2021-09-27  2021      9   27  0.0  25.0   7.4
2021-09-28  2021      9   28  0.0  16.6  11.3
2021-09-29  2021      9   29  0.0  25.5  13.0
2021-09-30  2021      9   30  0.0  28.9  11.4

[23984 rows x 6 columns]


...

/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Roque Saenz Peña aero
La diferencia de fechas es de 180

            year  month  day    pp  tmax  tmin
date                                          
1931-01-01  1931      1    1   1.0  32.8  26.6
1931-01-02  1931      1    2   0.0  33.3  20.9
1931-01-03  1931      1    3   0.0  35.7  22.8
1931-01-04  1931      1    4   0.0  37.5  24.5
1931-01-05  1931      1    5  47.0  36.0  24.5
...          ...    ...  ...   ...   ...   ...
2021-09-26  2021      9   26   0.0  34.4   9.5
2021-09-27  2021      9   27  32.0  36.3  15.0
2021-09-28  2021      9   28   3.0  29.0  20.5
2021-09-29  2021      9   29   0.0  22.1  19.3
2021-09-30  2021      9   30  24.0  34.9  18.0

[33146 rows x 6 columns]


/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Santa Rosa de Conlara aero
La diferencia de fechas es de 339

            year  month  day    pp  tmax  tmin
date                                          
2001-12-20  2001     12   20  15.0  35.8 -99.9
2001-12-21  2001     12   21   0.0 -99.9 -99.9
2001-12-22  2001     12   22   0.0 -99.9 -99.9
2001-12-23  2001     12   23   0.0 -99.9 -99.9
2001-12-24  2001     12   24   0.0 -99.9 -99.9
...          ...    ...  ...   ...   ...   ...
2021-09-26  2021      9   26   0.0  26.0   8.2
2021-09-27  2021      9   27   0.0  29.0   3.8
2021-09-28  2021      9   28   0.0  27.4   9.2
2021-09-29  2021      9   29   0.0  28.0   5.8
2021-09-30  2021      9   30   0.0  24.6  15.0

[7225 rows x 6 columns]
Las Breñas
La diferencia de fechas es de 0

            year  month  day    pp  tmax  tmin
date                                          
1967-01-01  1967      1    1   4.2  22.8  17.4
1967-01-02  1967      1    2  70.0  29.4  19.8
1967-01-03  1967      1    3  12.9  27.0  19.9
1967-01-04  1967      1

/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/dalia.panza/anaconda3/envs/env1/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u